# Computational Theory Assessment

In [1]:
import numpy as np

## Problem 1: Binary Words and Operations

In [ ]:
# Parity 
def parity(x, y, z):
    """
    Return the parity (XOR) of three 32-bit numbers.
    
    Each bit of the result is 1 if an odd number of the inputs
    have a 1 in that position, otherwise 0.
    """
    return np.uint32(x ^ y ^ z)

# Testing
print(parity(0, 0, 0))  # expect 0
print(parity(0, 0, 1))  # expect 1
print(parity(1, 1, 0))  # expect 0
print(parity(1, 1, 1))  # expect 1

## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords

# END